In [0]:
from pyspark.sql import functions as F

In [0]:
fact_weather_enriched_glm = (spark.table("hive_metastore.tfm.fact_weather"))

1. Substituir nulls por 0

In [0]:
numeric_cols = [c for c, t in fact_weather_enriched_glm.dtypes if t in ("int", "bigint", "float", "double")]

for c in numeric_cols:
    fact_weather_enriched_glm = fact_weather_enriched_glm.fillna({c: 0})

In [0]:
display(fact_weather_enriched_glm.limit(10))

weather_station_id,date,awnd,prcp,snow,snwd,tmax,tmin,wt01,wt02,wt03,wt04,wt05,wt06,wt08
USW00094728,2020-01-01,3.8,0.0,0.0,0.0,5.0,1.1,0,0,0,0,0,0,0
USW00094728,2020-01-02,2.4,0.0,0.0,0.0,9.4,0.6,0,0,0,0,0,0,0
USW00094728,2020-01-03,1.5,3.8,0.0,0.0,9.4,6.7,1,0,0,0,0,0,1
USW00094728,2020-01-04,2.0,6.9,0.0,0.0,10.6,5.0,1,1,0,0,0,0,0
USW00094728,2020-01-05,5.1,0.0,0.0,0.0,5.6,1.7,0,0,0,0,0,0,0
USW00094728,2020-01-06,2.7,1.0,5.0,0.0,7.2,0.0,1,0,0,0,0,0,0
USW00094728,2020-01-07,2.3,0.0,0.0,0.0,7.2,2.2,0,0,0,0,0,0,0
USW00094728,2020-01-08,4.3,0.0,0.0,0.0,5.6,-2.1,0,0,0,0,0,0,1
USW00094728,2020-01-09,2.0,0.0,0.0,0.0,1.1,-4.9,0,0,0,0,0,0,0
USW00094728,2020-01-10,2.8,0.0,0.0,0.0,12.8,0.6,0,0,0,0,0,0,0


2. Crear nueva columna de temperatura

In [0]:
fact_weather_enriched_glm = (
    fact_weather_enriched_glm
    .withColumn('trange', F.col('tmax') - F.col('tmin'))
)

3. Crear columna concatenada para el JOIN con las tablas de transporte

In [0]:
fact_weather_enriched_glm = (
    fact_weather_enriched_glm
        .withColumn(
            'weather_station_date',
            F.concat_ws('/', F.col('weather_station_id'), F.col('date'))
        )
)

In [0]:
display(fact_weather_enriched_glm.limit(10))

weather_station_id,date,awnd,prcp,snow,snwd,tmax,tmin,wt01,wt02,wt03,wt04,wt05,wt06,wt08,trange,weather_station_date
USW00094789,2020-01-01,28.437108480000006,0.0,0.0,0.0,5.555555555555555,1.1111111111111112,0,0,0,0,0,0,0,4.444444444444445,USW00094789/2020-01-01
USW00094789,2020-01-02,15.481889279999999,0.0,0.0,0.0,8.88888888888889,-0.5555555555555556,0,0,0,0,0,0,0,9.444444444444445,USW00094789/2020-01-02
USW00094789,2020-01-03,10.444642560000002,3.302,0.0,0.0,8.88888888888889,5.555555555555555,1,0,0,0,0,0,0,3.333333333333334,USW00094789/2020-01-03
USW00094789,2020-01-04,13.325368319999999,4.064,0.0,0.0,10.555555555555555,6.111111111111112,1,1,0,0,0,0,0,4.444444444444444,USW00094789/2020-01-04
USW00094789,2020-01-05,34.55261568,0.0,0.0,0.0,9.444444444444445,1.6666666666666667,0,0,0,0,0,0,0,7.777777777777778,USW00094789/2020-01-05
USW00094789,2020-01-06,22.32160128,0.7619999999999999,0.3,0.0,7.222222222222222,0.0,1,0,0,0,0,0,0,7.222222222222222,USW00094789/2020-01-06
USW00094789,2020-01-07,13.679424000000001,0.7619999999999999,0.0,0.0,6.666666666666667,1.6666666666666667,0,0,0,0,0,0,0,5.0,USW00094789/2020-01-07
USW00094789,2020-01-08,33.474355200000005,0.0,0.0,0.0,6.111111111111112,-2.2222222222222223,0,0,0,0,0,0,0,8.333333333333334,USW00094789/2020-01-08
USW00094789,2020-01-09,17.99246592,0.0,0.0,0.0,1.1111111111111112,-4.444444444444445,0,0,0,0,0,0,0,5.555555555555555,USW00094789/2020-01-09
USW00094789,2020-01-10,12.601163520000002,0.0,0.0,0.0,10.555555555555555,0.0,0,0,0,0,0,0,0,10.555555555555555,USW00094789/2020-01-10


4. Guardar tabla

In [0]:
fact_weather_enriched_glm.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("tfm.fact_weather_enriched_glm")